**About :** Trains XGBoost models.

**TODO**:
- better neg sampling technique ??

In [ ]:
cd ../src

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0"

In [ ]:
import os
import gc
import cudf
import json
import glob
import numba
import xgboost
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

from tqdm import tqdm
from pandarallel import pandarallel
from numerize.numerize import numerize

warnings.simplefilter(action="ignore", category=FutureWarning)
pd.options.display.max_columns = 500
pd.options.display.max_rows = 500
pandarallel.initialize(nb_workers=32, progress_bar=False)

In [ ]:
from params import *

from utils.metrics import get_coverage
from utils.plot import plot_importances
from utils.load import *
from utils.logger import save_config, prepare_log_folder, create_logger, init_neptune
from utils.torch import seed_everything

from training.xgb import kfold, optimize

In [ ]:
import matplotlib
matplotlib.use('Agg')

### Load

In [ ]:
# VERSION = "clicks_cv3-tv5.10"
# VERSION = "cv3-tv5.10"
# VERSION = "cv6-tv6.11"
# TEST_VERSION = "cv3-tv5.10"

VERSION = "cv7-tv5.11"
TEST_VERSION = "cv7-tv5.11"

#### Train data
- neg sampling could use candidates from lower versions

In [ ]:
POS_RATIO = 0.2
TARGET = "gt_carts"   # "gt_clicks", "gt_carts", "gt_orders"

In [ ]:
PROBS_PATHS = {
    "gt_orders": "../logs/2023-01-14/9/",
    "gt_carts": "../logs/2023-01-14/8/",
}

In [ ]:
# if TARGET != "gt_clicks":
#     REGEX = f"../output/features/fts_val_{VERSION}_{TARGET}/*"
# else:

REGEX = f"../output/features/fts_val_{VERSION}/*"
len(glob.glob(REGEX))

In [ ]:
EXTRA_REGEX = f"../output/features/fts_extra_{VERSION}/*"
len(glob.glob(EXTRA_REGEX))

In [ ]:
TEST_REGEX = f"../output/features/fts_test_{TEST_VERSION}/*"
len(glob.glob(TEST_REGEX))

In [ ]:
FEATURES = [  # REMOVE CORRELATED
    'clicks_popularity_w_pos-log', 'clicks_popularity_w_type-163', 'clicks_popularity_w_lastday', 'clicks_popularity_w_recsys', 
    'carts_popularity_w_pos-log', 'carts_popularity_w_type-163', 'carts_popularity_w_lastday', 'carts_popularity_w_recsys', 
    'orders_popularity_w_pos-log', 'orders_popularity_w_type-163', 'orders_popularity_w_lastday', 'orders_popularity_w_recsys', 
    'clicks_popularity_w_pos-log_w', 'clicks_popularity_w_type-163_w', 'clicks_popularity_w_recsys_w', 
    'carts_popularity_w_pos-log_w', 'carts_popularity_w_type-163_w', 'carts_popularity_w_recsys_w', 
    'orders_popularity_w_pos-log_w', 'orders_popularity_w_type-163_w', 'orders_popularity_w_recsys_w',
    'w_pos-log', 'w_type-163', 'w_lastday', 'w_time', 'w_recsys',
    'matrix_123_temporal_20_mean', 'matrix_123_temporal_20_sum', 'matrix_123_temporal_20_max', 'matrix_123_temporal_20_pos-log_mean', 'matrix_123_temporal_20_pos-log_sum', 'matrix_123_temporal_20_pos-log_max', 'matrix_123_temporal_20_type-163_mean', 'matrix_123_temporal_20_type-163_sum', 'matrix_123_temporal_20_type-163_max', 'matrix_123_temporal_20_lastday_mean', 'matrix_123_temporal_20_lastday_sum', 'matrix_123_temporal_20_lastday_max', 'matrix_123_temporal_20_time_mean', 'matrix_123_temporal_20_time_sum', 'matrix_123_temporal_20_time_max', 'matrix_123_temporal_20_recsys_mean', 'matrix_123_temporal_20_recsys_sum', 'matrix_123_temporal_20_recsys_max',
    'matrix_123_type136_20_mean', 'matrix_123_type136_20_sum', 'matrix_123_type136_20_max', 'matrix_123_type136_20_pos-log_mean', 'matrix_123_type136_20_pos-log_sum', 'matrix_123_type136_20_pos-log_max', 'matrix_123_type136_20_type-163_mean', 'matrix_123_type136_20_type-163_sum', 'matrix_123_type136_20_type-163_max', 'matrix_123_type136_20_lastday_mean', 'matrix_123_type136_20_lastday_sum', 'matrix_123_type136_20_lastday_max', 'matrix_123_type136_20_time_mean', 'matrix_123_type136_20_time_sum', 'matrix_123_type136_20_time_max', 'matrix_123_type136_20_recsys_mean', 'matrix_123_type136_20_recsys_sum', 'matrix_123_type136_20_recsys_max',
    'matrix_12__20_mean', 'matrix_12__20_sum', 'matrix_12__20_max', 'matrix_12__20_pos-log_mean', 'matrix_12__20_pos-log_sum', 'matrix_12__20_pos-log_max', 'matrix_12__20_type-163_mean', 'matrix_12__20_type-163_sum', 'matrix_12__20_type-163_max', 'matrix_12__20_lastday_mean', 'matrix_12__20_lastday_sum', 'matrix_12__20_lastday_max', 'matrix_12__20_time_mean', 'matrix_12__20_time_sum', 'matrix_12__20_time_max', 'matrix_12__20_recsys_mean', 'matrix_12__20_recsys_sum', 'matrix_12__20_recsys_max',
    'matrix_123_type0.590.5_20_mean', 'matrix_123_type0.590.5_20_sum', 'matrix_123_type0.590.5_20_max', 'matrix_123_type0.590.5_20_pos-log_mean', 'matrix_123_type0.590.5_20_pos-log_sum', 'matrix_123_type0.590.5_20_pos-log_max', 'matrix_123_type0.590.5_20_type-163_mean', 'matrix_123_type0.590.5_20_type-163_sum', 'matrix_123_type0.590.5_20_type-163_max', 'matrix_123_type0.590.5_20_lastday_mean', 'matrix_123_type0.590.5_20_lastday_sum', 'matrix_123_type0.590.5_20_lastday_max', 'matrix_123_type0.590.5_20_time_mean', 'matrix_123_type0.590.5_20_time_sum', 'matrix_123_type0.590.5_20_time_max', 'matrix_123_type0.590.5_20_recsys_mean', 'matrix_123_type0.590.5_20_recsys_sum', 'matrix_123_type0.590.5_20_recsys_max',
    'matrix_cpu-90_mean', 'matrix_cpu-90_sum', 'matrix_cpu-90_max', 'matrix_cpu-90_pos-log_mean', 'matrix_cpu-90_pos-log_sum', 'matrix_cpu-90_pos-log_max', 'matrix_cpu-90_type-163_mean', 'matrix_cpu-90_type-163_sum', 'matrix_cpu-90_type-163_max', 'matrix_cpu-90_lastday_mean', 'matrix_cpu-90_lastday_sum', 'matrix_cpu-90_lastday_max', 'matrix_cpu-90_time_mean', 'matrix_cpu-90_time_sum', 'matrix_cpu-90_time_max', 'matrix_cpu-90_recsys_mean', 'matrix_cpu-90_recsys_sum', 'matrix_cpu-90_recsys_max',
    'matrix_cpu-95_mean', 'matrix_cpu-95_sum', 'matrix_cpu-95_max', 'matrix_cpu-95_pos-log_mean', 'matrix_cpu-95_pos-log_sum', 'matrix_cpu-95_pos-log_max', 'matrix_cpu-95_type-163_mean', 'matrix_cpu-95_type-163_sum', 'matrix_cpu-95_type-163_max', 'matrix_cpu-95_lastday_mean', 'matrix_cpu-95_lastday_sum', 'matrix_cpu-95_lastday_max', 'matrix_cpu-95_time_mean', 'matrix_cpu-95_time_sum', 'matrix_cpu-95_time_max', 'matrix_cpu-95_recsys_mean', 'matrix_cpu-95_recsys_sum', 'matrix_cpu-95_recsys_max',
    'matrix_cpu-99_mean', 'matrix_cpu-99_sum', 'matrix_cpu-99_max', 'matrix_cpu-99_pos-log_mean', 'matrix_cpu-99_pos-log_sum', 'matrix_cpu-99_pos-log_max', 'matrix_cpu-99_type-163_mean', 'matrix_cpu-99_type-163_sum', 'matrix_cpu-99_type-163_max', 'matrix_cpu-99_lastday_mean', 'matrix_cpu-99_lastday_sum', 'matrix_cpu-99_lastday_max', 'matrix_cpu-99_time_mean', 'matrix_cpu-99_time_sum', 'matrix_cpu-99_time_max', 'matrix_cpu-99_recsys_mean', 'matrix_cpu-99_recsys_sum', 'matrix_cpu-99_recsys_max',
    'matrix_gpu-116_mean', 'matrix_gpu-116_sum', 'matrix_gpu-116_max', 'matrix_gpu-116_pos-log_mean', 'matrix_gpu-116_pos-log_sum', 'matrix_gpu-116_pos-log_max', 'matrix_gpu-116_type-163_mean', 'matrix_gpu-116_type-163_sum', 'matrix_gpu-116_type-163_max', 'matrix_gpu-116_lastday_mean', 'matrix_gpu-116_lastday_sum', 'matrix_gpu-116_lastday_max', 'matrix_gpu-116_time_mean', 'matrix_gpu-116_time_sum', 'matrix_gpu-116_time_max', 'matrix_gpu-116_recsys_mean', 'matrix_gpu-116_recsys_sum', 'matrix_gpu-116_recsys_max',
    'matrix_gpu-115_mean', 'matrix_gpu-115_sum', 'matrix_gpu-115_max', 'matrix_gpu-115_pos-log_mean', 'matrix_gpu-115_pos-log_sum', 'matrix_gpu-115_pos-log_max', 'matrix_gpu-115_type-163_mean', 'matrix_gpu-115_type-163_sum', 'matrix_gpu-115_type-163_max', 'matrix_gpu-115_lastday_mean', 'matrix_gpu-115_lastday_sum', 'matrix_gpu-115_lastday_max', 'matrix_gpu-115_time_mean', 'matrix_gpu-115_time_sum', 'matrix_gpu-115_time_max', 'matrix_gpu-115_recsys_mean', 'matrix_gpu-115_recsys_sum', 'matrix_gpu-115_recsys_max',
    'matrix_gpu-93_mean', 'matrix_gpu-93_sum', 'matrix_gpu-93_max', 'matrix_gpu-93_pos-log_mean', 'matrix_gpu-93_pos-log_sum', 'matrix_gpu-93_pos-log_max', 'matrix_gpu-93_type-163_mean', 'matrix_gpu-93_type-163_sum', 'matrix_gpu-93_type-163_max', 'matrix_gpu-93_lastday_mean', 'matrix_gpu-93_lastday_sum', 'matrix_gpu-93_lastday_max', 'matrix_gpu-93_time_mean', 'matrix_gpu-93_time_sum', 'matrix_gpu-93_time_max', 'matrix_gpu-93_recsys_mean', 'matrix_gpu-93_recsys_sum', 'matrix_gpu-93_recsys_max',
    'matrix_gpu-217_mean', 'matrix_gpu-217_sum', 'matrix_gpu-217_max', 'matrix_gpu-217_pos-log_mean', 'matrix_gpu-217_pos-log_sum', 'matrix_gpu-217_pos-log_max', 'matrix_gpu-217_type-163_mean', 'matrix_gpu-217_type-163_sum', 'matrix_gpu-217_type-163_max', 'matrix_gpu-217_lastday_mean', 'matrix_gpu-217_lastday_sum', 'matrix_gpu-217_lastday_max', 'matrix_gpu-217_time_mean', 'matrix_gpu-217_time_sum', 'matrix_gpu-217_time_max', 'matrix_gpu-217_recsys_mean', 'matrix_gpu-217_recsys_sum', 'matrix_gpu-217_recsys_max',
    'matrix_gpu-226_mean','matrix_gpu-226_sum','matrix_gpu-226_max','matrix_gpu-226_pos-log_mean','matrix_gpu-226_pos-log_sum','matrix_gpu-226_pos-log_max','matrix_gpu-226_type-163_mean','matrix_gpu-226_type-163_sum','matrix_gpu-226_type-163_max','matrix_gpu-226_lastday_mean','matrix_gpu-226_lastday_sum','matrix_gpu-226_lastday_max','matrix_gpu-226_time_mean','matrix_gpu-226_time_sum','matrix_gpu-226_time_max','matrix_gpu-226_recsys_mean','matrix_gpu-226_recsys_sum','matrix_gpu-226_recsys_max',
    'matrix_gpu-232_mean', 'matrix_gpu-232_sum', 'matrix_gpu-232_max', 'matrix_gpu-232_pos-log_mean', 'matrix_gpu-232_pos-log_sum', 'matrix_gpu-232_pos-log_max', 'matrix_gpu-232_type-163_mean', 'matrix_gpu-232_type-163_sum', 'matrix_gpu-232_type-163_max', 'matrix_gpu-232_lastday_mean', 'matrix_gpu-232_lastday_sum', 'matrix_gpu-232_lastday_max', 'matrix_gpu-232_time_mean', 'matrix_gpu-232_time_sum', 'matrix_gpu-232_time_max', 'matrix_gpu-232_recsys_mean', 'matrix_gpu-232_recsys_sum', 'matrix_gpu-232_recsys_max',
    'matrix_gpu-239_mean', 'matrix_gpu-239_sum', 'matrix_gpu-239_max', 'matrix_gpu-239_pos-log_mean', 'matrix_gpu-239_pos-log_sum', 'matrix_gpu-239_pos-log_max', 'matrix_gpu-239_type-163_mean', 'matrix_gpu-239_type-163_sum', 'matrix_gpu-239_type-163_max', 'matrix_gpu-239_lastday_mean', 'matrix_gpu-239_lastday_sum', 'matrix_gpu-239_lastday_max', 'matrix_gpu-239_time_mean', 'matrix_gpu-239_time_sum', 'matrix_gpu-239_time_max', 'matrix_gpu-239_recsys_mean', 'matrix_gpu-239_recsys_sum', 'matrix_gpu-239_recsys_max',
    'matrix_gpu-700_mean', 'matrix_gpu-700_sum', 'matrix_gpu-700_max', 'matrix_gpu-700_pos-log_mean', 'matrix_gpu-700_pos-log_sum', 'matrix_gpu-700_pos-log_max', 'matrix_gpu-700_type-163_mean', 'matrix_gpu-700_type-163_sum', 'matrix_gpu-700_type-163_max', 'matrix_gpu-700_lastday_mean', 'matrix_gpu-700_lastday_sum', 'matrix_gpu-700_lastday_max', 'matrix_gpu-700_time_mean', 'matrix_gpu-700_time_sum', 'matrix_gpu-700_time_max', 'matrix_gpu-700_recsys_mean', 'matrix_gpu-700_recsys_sum', 'matrix_gpu-700_recsys_max',
    'matrix_gpu-701_mean', 'matrix_gpu-701_sum', 'matrix_gpu-701_max', 'matrix_gpu-701_pos-log_mean', 'matrix_gpu-701_pos-log_sum', 'matrix_gpu-701_pos-log_max', 'matrix_gpu-701_type-163_mean', 'matrix_gpu-701_type-163_sum', 'matrix_gpu-701_type-163_max', 'matrix_gpu-701_lastday_mean', 'matrix_gpu-701_lastday_sum', 'matrix_gpu-701_lastday_max', 'matrix_gpu-701_time_mean', 'matrix_gpu-701_time_sum', 'matrix_gpu-701_time_max', 'matrix_gpu-701_recsys_mean', 'matrix_gpu-701_recsys_sum', 'matrix_gpu-701_recsys_max',
    'candidate_clicks_before', 'candidate_carts_before', 'candidate_orders_before', 'candidate_*_before', 'n_views', 'n_clicks', 'n_carts', 'n_orders',
    'clicks_popularity_w_pos-log_rank', 'clicks_popularity_w_type-163_rank', 'clicks_popularity_w_lastday_rank', 'clicks_popularity_w_time_rank', 'clicks_popularity_w_recsys_rank', 'carts_popularity_w_pos-log_rank', 'carts_popularity_w_type-163_rank', 'carts_popularity_w_lastday_rank', 'carts_popularity_w_time_rank', 'carts_popularity_w_recsys_rank', 'orders_popularity_w_pos-log_rank', 'orders_popularity_w_type-163_rank', 'orders_popularity_w_lastday_rank', 'orders_popularity_w_time_rank', 'orders_popularity_w_recsys_rank',
    'clicks_popularity_w_pos-log_w_rank', 'clicks_popularity_w_type-163_w_rank', 'clicks_popularity_w_lastday_w_rank', 'clicks_popularity_w_time_w_rank', 'clicks_popularity_w_recsys_w_rank', 'carts_popularity_w_pos-log_w_rank', 'carts_popularity_w_type-163_w_rank', 'carts_popularity_w_lastday_w_rank', 'carts_popularity_w_time_w_rank', 'carts_popularity_w_recsys_w_rank', 'orders_popularity_w_pos-log_w_rank', 'orders_popularity_w_type-163_w_rank', 'orders_popularity_w_lastday_w_rank', 'orders_popularity_w_time_w_rank', 'orders_popularity_w_recsys_w_rank',
    'w_pos-log_rank', 'w_type-163_rank', 'w_lastday_rank', 'w_time_rank', 'w_recsys_rank',
    'matrix_123_temporal_20_mean_rank', 'matrix_123_temporal_20_pos-log_mean_rank', 'matrix_123_temporal_20_type-163_mean_rank', 'matrix_123_temporal_20_lastday_mean_rank', 'matrix_123_temporal_20_time_mean_rank', 'matrix_123_temporal_20_recsys_mean_rank', 'matrix_123_type136_20_mean_rank', 'matrix_123_type136_20_pos-log_mean_rank', 'matrix_123_type136_20_type-163_mean_rank', 'matrix_123_type136_20_lastday_mean_rank', 'matrix_123_type136_20_time_mean_rank', 'matrix_123_type136_20_recsys_mean_rank', 
    'matrix_12__20_mean_rank', 'matrix_12__20_pos-log_mean_rank', 'matrix_12__20_type-163_mean_rank', 'matrix_12__20_lastday_mean_rank', 'matrix_12__20_time_mean_rank', 'matrix_12__20_recsys_mean_rank', 'matrix_123_type0.590.5_20_mean_rank', 'matrix_123_type0.590.5_20_pos-log_mean_rank', 'matrix_123_type0.590.5_20_type-163_mean_rank', 'matrix_123_type0.590.5_20_lastday_mean_rank', 'matrix_123_type0.590.5_20_time_mean_rank', 'matrix_123_type0.590.5_20_recsys_mean_rank',
    'matrix_cpu-90_mean_rank', 'matrix_cpu-90_pos-log_mean_rank', 'matrix_cpu-90_type-163_mean_rank', 'matrix_cpu-90_lastday_mean_rank', 'matrix_cpu-90_time_mean_rank', 'matrix_cpu-90_recsys_mean_rank', 'matrix_cpu-95_mean_rank', 'matrix_cpu-95_pos-log_mean_rank', 'matrix_cpu-95_type-163_mean_rank', 'matrix_cpu-95_lastday_mean_rank', 'matrix_cpu-95_time_mean_rank', 'matrix_cpu-95_recsys_mean_rank', 'matrix_cpu-99_mean_rank', 'matrix_cpu-99_pos-log_mean_rank', 'matrix_cpu-99_type-163_mean_rank', 'matrix_cpu-99_lastday_mean_rank', 'matrix_cpu-99_time_mean_rank', 'matrix_cpu-99_recsys_mean_rank',
    'matrix_gpu-116_mean_rank', 'matrix_gpu-116_pos-log_mean_rank', 'matrix_gpu-116_type-163_mean_rank', 'matrix_gpu-116_lastday_mean_rank', 'matrix_gpu-116_time_mean_rank', 'matrix_gpu-116_recsys_mean_rank', 'matrix_gpu-115_mean_rank', 'matrix_gpu-115_pos-log_mean_rank', 'matrix_gpu-115_type-163_mean_rank', 'matrix_gpu-115_lastday_mean_rank', 'matrix_gpu-115_time_mean_rank', 'matrix_gpu-115_recsys_mean_rank', 'matrix_gpu-93_mean_rank', 'matrix_gpu-93_pos-log_mean_rank', 'matrix_gpu-93_type-163_mean_rank', 'matrix_gpu-93_lastday_mean_rank', 'matrix_gpu-93_time_mean_rank', 'matrix_gpu-93_recsys_mean_rank',
    'matrix_gpu-217_mean_rank', 'matrix_gpu-217_pos-log_mean_rank', 'matrix_gpu-217_type-163_mean_rank', 'matrix_gpu-217_lastday_mean_rank', 'matrix_gpu-217_time_mean_rank', 'matrix_gpu-217_recsys_mean_rank', 'matrix_gpu-226_mean_rank', 'matrix_gpu-226_pos-log_mean_rank', 'matrix_gpu-226_type-163_mean_rank', 'matrix_gpu-226_lastday_mean_rank', 'matrix_gpu-226_time_mean_rank', 'matrix_gpu-226_recsys_mean_rank', 'matrix_gpu-232_mean_rank', 'matrix_gpu-232_pos-log_mean_rank', 'matrix_gpu-232_type-163_mean_rank', 'matrix_gpu-232_lastday_mean_rank', 'matrix_gpu-232_time_mean_rank', 'matrix_gpu-232_recsys_mean_rank',
    'matrix_gpu-239_mean_rank', 'matrix_gpu-239_pos-log_mean_rank', 'matrix_gpu-239_type-163_mean_rank', 'matrix_gpu-239_lastday_mean_rank', 'matrix_gpu-239_time_mean_rank', 'matrix_gpu-239_recsys_mean_rank', 'matrix_gpu-700_mean_rank', 'matrix_gpu-700_pos-log_mean_rank', 'matrix_gpu-700_type-163_mean_rank', 'matrix_gpu-700_lastday_mean_rank', 'matrix_gpu-700_time_mean_rank', 'matrix_gpu-700_recsys_mean_rank', 'matrix_gpu-701_mean_rank', 'matrix_gpu-701_pos-log_mean_rank', 'matrix_gpu-701_type-163_mean_rank', 'matrix_gpu-701_lastday_mean_rank', 'matrix_gpu-701_time_mean_rank', 'matrix_gpu-701_recsys_mean_rank',
]

In [ ]:
FEATURES += [
    'popularity_week_clicks','popularity_day_clicks','popularity_hour_clicks','popularity_hour/day_clicks','popularity_day/week_clicks','popularity_week_carts','popularity_day_carts','popularity_hour_carts','popularity_hour/day_carts','popularity_day/week_carts','popularity_week_orders','popularity_day_orders','popularity_hour_orders','popularity_hour/day_orders','popularity_day/week_orders',
    'embed_1-9_64_cartbuy_last_0', 'embed_1-9_64_cartbuy_last_1', 'embed_1-9_64_cartbuy_last_2', 'embed_1-9_64_cartbuy_last_3', 'embed_1-9_64_cartbuy_last_4', 'embed_1-9_64_cartbuy_pos-log_mean', 'embed_1-9_64_cartbuy_pos-log_sum', 'embed_1-9_64_cartbuy_pos-log_max', 'embed_1-9_64_cartbuy_type-163_mean', 'embed_1-9_64_cartbuy_type-163_sum', 'embed_1-9_64_cartbuy_type-163_max', 'embed_1-9_64_cartbuy_lastday_mean', 'embed_1-9_64_cartbuy_lastday_sum', 'embed_1-9_64_cartbuy_lastday_max', 'embed_1-9_64_cartbuy_time_mean', 'embed_1-9_64_cartbuy_time_sum', 'embed_1-9_64_cartbuy_time_max', 'embed_1-9_64_cartbuy_recsys_mean', 'embed_1-9_64_cartbuy_recsys_sum', 'embed_1-9_64_cartbuy_recsys_max',
    'embed_1_64_last_0', 'embed_1_64_last_1', 'embed_1_64_last_2', 'embed_1_64_last_3', 'embed_1_64_last_4', 'embed_1_64_pos-log_mean', 'embed_1_64_pos-log_sum', 'embed_1_64_pos-log_max', 'embed_1_64_type-163_mean', 'embed_1_64_type-163_sum', 'embed_1_64_type-163_max', 'embed_1_64_lastday_mean', 'embed_1_64_lastday_sum', 'embed_1_64_lastday_max', 'embed_1_64_time_mean', 'embed_1_64_time_sum', 'embed_1_64_time_max', 'embed_1_64_recsys_mean', 'embed_1_64_recsys_sum', 'embed_1_64_recsys_max',
    'embed_1-5_64_last_0', 'embed_1-5_64_last_1', 'embed_1-5_64_last_2', 'embed_1-5_64_last_3', 'embed_1-5_64_last_4', 'embed_1-5_64_pos-log_mean', 'embed_1-5_64_pos-log_sum', 'embed_1-5_64_pos-log_max', 'embed_1-5_64_type-163_mean', 'embed_1-5_64_type-163_sum', 'embed_1-5_64_type-163_max', 'embed_1-5_64_lastday_mean', 'embed_1-5_64_lastday_sum', 'embed_1-5_64_lastday_max', 'embed_1-5_64_time_mean', 'embed_1-5_64_time_sum', 'embed_1-5_64_time_max', 'embed_1-5_64_recsys_mean', 'embed_1-5_64_recsys_sum', 'embed_1-5_64_recsys_max',
]


In [ ]:
FEATURES += [
    'popularity_week_clicks_rank', 'popularity_day_clicks_rank', 'popularity_hour_clicks_rank', 'popularity_hour/day_clicks_rank', 'popularity_day/week_clicks_rank', 'popularity_week_carts_rank', 'popularity_day_carts_rank', 'popularity_hour_carts_rank', 'popularity_hour/day_carts_rank', 'popularity_day/week_carts_rank', 'popularity_week_orders_rank', 'popularity_day_orders_rank', 'popularity_hour_orders_rank', 'popularity_hour/day_orders_rank', 'popularity_day/week_orders_rank',
    'embed_1-9_64_cartbuy_last_0_rank', 'embed_1-9_64_cartbuy_last_1_rank', 'embed_1-9_64_cartbuy_last_2_rank', 'embed_1-9_64_cartbuy_last_3_rank', 'embed_1-9_64_cartbuy_last_4_rank', 'embed_1-9_64_cartbuy_pos-log_mean_rank', 'embed_1-9_64_cartbuy_type-163_mean_rank', 'embed_1-9_64_cartbuy_lastday_mean_rank', 'embed_1-9_64_cartbuy_time_mean_rank', 'embed_1-9_64_cartbuy_recsys_mean_rank', 'embed_1_64_last_0_rank', 'embed_1_64_last_1_rank', 'embed_1_64_last_2_rank', 'embed_1_64_last_3_rank', 'embed_1_64_last_4_rank', 'embed_1_64_pos-log_mean_rank', 'embed_1_64_type-163_mean_rank', 'embed_1_64_lastday_mean_rank', 'embed_1_64_time_mean_rank', 'embed_1_64_recsys_mean_rank', 'embed_1-5_64_last_0_rank', 'embed_1-5_64_last_1_rank', 'embed_1-5_64_last_2_rank', 'embed_1-5_64_last_3_rank', 'embed_1-5_64_last_4_rank', 'embed_1-5_64_pos-log_mean_rank', 'embed_1-5_64_type-163_mean_rank', 'embed_1-5_64_lastday_mean_rank', 'embed_1-5_64_time_mean_rank', 'embed_1-5_64_recsys_mean_rank'
]

In [ ]:
FEATURES += [
    'w2v_sim_1', 'w2v_sim_2', 'w2v_sim_3', 'w2v_sim_wgt_1', 'w2v_sim_wgt_2', 'w2v_sim_last', 'w2v_sim_type_1', 'w2v_sim_1_rank', 'w2v_sim_2_rank', 'w2v_sim_3_rank', 'w2v_sim_wgt_1_rank', 'w2v_sim_wgt_2_rank', 'w2v_sim_last_rank', 'w2v_sim_type_1_rank',
    'matrix_gpu-155_mean', 'matrix_gpu-155_sum', 'matrix_gpu-155_max', 'matrix_gpu-155_pos-log_mean', 'matrix_gpu-155_pos-log_sum', 'matrix_gpu-155_pos-log_max', 'matrix_gpu-155_type-163_mean', 'matrix_gpu-155_type-163_sum', 'matrix_gpu-155_type-163_max', 'matrix_gpu-155_lastday_mean', 'matrix_gpu-155_lastday_sum', 'matrix_gpu-155_lastday_max', 'matrix_gpu-155_time_mean', 'matrix_gpu-155_time_sum', 'matrix_gpu-155_time_max', 'matrix_gpu-155_recsys_mean', 'matrix_gpu-155_recsys_sum', 'matrix_gpu-155_recsys_max',
    'matrix_gpu-157_mean', 'matrix_gpu-157_sum', 'matrix_gpu-157_max', 'matrix_gpu-157_pos-log_mean', 'matrix_gpu-157_pos-log_sum', 'matrix_gpu-157_pos-log_max', 'matrix_gpu-157_type-163_mean', 'matrix_gpu-157_type-163_sum', 'matrix_gpu-157_type-163_max', 'matrix_gpu-157_lastday_mean', 'matrix_gpu-157_lastday_sum', 'matrix_gpu-157_lastday_max', 'matrix_gpu-157_time_mean', 'matrix_gpu-157_time_sum', 'matrix_gpu-157_time_max', 'matrix_gpu-157_recsys_mean', 'matrix_gpu-157_recsys_sum', 'matrix_gpu-157_recsys_max',
    'matrix_gpu-155_mean_rank', 'matrix_gpu-155_pos-log_mean_rank', 'matrix_gpu-155_type-163_mean_rank', 'matrix_gpu-155_lastday_mean_rank', 'matrix_gpu-155_time_mean_rank', 'matrix_gpu-155_recsys_mean_rank', 'matrix_gpu-157_mean_rank', 'matrix_gpu-157_pos-log_mean_rank', 'matrix_gpu-157_type-163_mean_rank', 'matrix_gpu-157_lastday_mean_rank', 'matrix_gpu-157_time_mean_rank', 'matrix_gpu-157_recsys_mean_rank', 
]

In [ ]:
len(FEATURES)

In [ ]:
TO_REMOVE = []
TO_REMOVE += [f for f in FEATURES if "popularity_w_time" in f]
TO_REMOVE += [f for f in FEATURES if "popularity_w_lastday_w" in f]

FEATURES = [f for f in FEATURES if f not in TO_REMOVE]

In [ ]:
len(FEATURES)

In [ ]:
# df_train = cudf.from_pandas(df_train)
# corr = df_train[FEATURES].corr()
# corr = corr.to_pandas()
# corr = corr.values

# mask = np.zeros_like(corr, dtype=bool)
# mask[np.triu_indices_from(mask)] = True
# corr[mask] = 0

In [ ]:
# TH = 0.99

# for i in range(len(corr)):
#     for j in range(len(corr)):
#         if corr[i, j] > TH:
#             if FEATURES[i] in TO_REMOVE or FEATURES[j] in TO_REMOVE:
#                 continue
#             print(FEATURES[i], FEATURES[j], f'{corr[i, j] :.3f}')

In [ ]:
# df = cudf.read_parquet(glob.glob(REGEX)[0])
# df = df.rename(columns={"clicks_popularity_w_pos-log_rank" : "clicks_popularity_w_pos-log_rank_ref"})
# df = cudf.read_parquet(glob.glob(TEST_REGEX)[0])

# from data.fe import add_rank_feature
# for c in ['clicks_popularity_w_pos-log', 'clicks_popularity_w_type-163', 'clicks_popularity_w_lastday']:
#     if c + "_rank" not in df.columns:
#         print(f'Add rank ft for {c}')
#         df = df.reset_index(drop=True)
#         add_rank_feature(df, c)
# (df['clicks_popularity_w_pos-log_rank'] == df['clicks_popularity_w_pos-log_rank_ref']).all()

# for f in tqdm(glob.glob(TEST_REGEX)):
#     dft = cudf.read_parquet(f, columns=['clicks_popularity_w_pos-log_rank'])

### Params

In [ ]:
PARAMS = {
    "xgb":
    {
        "learning_rate": 0.01,
        'max_depth': 8,
        "subsample": 0.9,  # 0.7 / 0.8 / O.9
        'colsample_bytree': 0.7,  # 0.7 / 0.8 / 0.9
        'reg_alpha': 0.01,
        'reg_lambda': 0.1,
        "min_child_weight": 0,
#         "gamma": 0.01,
        'scale_pos_weight': 1,
        'eval_metric': 'auc',
        'objective': 'binary:logistic',  # 'binary:logistic',
        'tree_method':'gpu_hist',
        'predictor':'gpu_predictor',
        "random_state": 42,
    },
}

In [ ]:
TO_REMOVE = [
    'popularity_week_orders', 'candidate_*_before', 'popularity_week_carts', 'popularity_week_clicks_rank', 'popularity_week_orders_rank', 'popularity_week_clicks', 'popularity_week_carts_rank', 'matrix_gpu-226_lastday_max', 'matrix_cpu-90_lastday_sum', 'matrix_cpu-90_lastday_max', 
    'matrix_12__20_lastday_max', 'matrix_gpu-700_sum', 'matrix_123_type136_20_lastday_max', 'matrix_12__20_time_sum', 'matrix_gpu-226_lastday_sum', 'matrix_123_type136_20_lastday_sum', 'matrix_12__20_lastday_sum', 'matrix_gpu-217_lastday_max', 'matrix_cpu-90_time_sum', 'matrix_cpu-90_max', 
    'matrix_gpu-217_lastday_sum', 'matrix_cpu-90_type-163_sum', 'matrix_123_type136_20_time_sum', 'matrix_12__20_type-163_sum', 'matrix_12__20_pos-log_sum',
    'matrix_cpu-90_lastday_mean', 'matrix_gpu-226_max', 'matrix_cpu-90_pos-log_sum', 'popularity_hour_orders', 'matrix_12__20_pos-log_max', 'matrix_gpu-226_time_sum', 'matrix_12__20_time_mean', 'matrix_cpu-90_pos-log_max', 'matrix_cpu-99_time_sum', 'matrix_123_type0.590.5_20_time_sum', 
    'matrix_123_type136_20_type-163_max', 'matrix_gpu-701_lastday_max', 'matrix_gpu-217_sum', 'matrix_gpu-226_sum', 'matrix_123_type136_20_time_mean', 'matrix_123_type136_20_type-163_sum', 'matrix_cpu-99_lastday_sum', 'matrix_123_type0.590.5_20_lastday_max', 'matrix_cpu-90_type-163_max', 
    'matrix_gpu-226_type-163_sum', 'matrix_12__20_sum', 'matrix_gpu-217_time_sum', 'popularity_hour/day_orders', 'matrix_12__20_type-163_max', 'matrix_123_type136_20_pos-log_sum', 'matrix_gpu-700_lastday_max', 'matrix_123_type136_20_lastday_mean', 'matrix_gpu-700_max', 'matrix_123_type136_20_time_max',
    'matrix_cpu-90_time_max', 'matrix_cpu-90_sum', 'matrix_12__20_lastday_mean', 'matrix_gpu-226_pos-log_sum', 'matrix_12__20_max',
    'matrix_cpu-99_lastday_max', 'matrix_cpu-95_lastday_max', 'matrix_cpu-90_time_mean', 'matrix_gpu-217_pos-log_sum', 'matrix_12__20_type-163_mean', 'matrix_123_temporal_20_time_sum', 'matrix_cpu-90_pos-log_mean', 'matrix_123_type136_20_pos-log_max', 'matrix_123_type136_20_sum', 
    'matrix_gpu-226_lastday_mean', 'matrix_gpu-701_time_sum', 'matrix_gpu-700_pos-log_sum', 'matrix_cpu-95_lastday_sum', 'matrix_12__20_time_max', 'matrix_gpu-217_type-163_max', 'matrix_cpu-95_time_mean', 'matrix_123_type0.590.5_20_lastday_sum', 'matrix_123_type0.590.5_20_type-163_max', 
    'matrix_gpu-226_time_mean', 'matrix_gpu-701_lastday_sum', 'matrix_123_temporal_20_lastday_max', 'matrix_123_type0.590.5_20_pos-log_sum', 'matrix_cpu-95_time_sum', 'matrix_123_type0.590.5_20_type-163_sum', 'matrix_cpu-99_lastday_mean', 'matrix_cpu-99_time_mean', 'matrix_12__20_pos-log_mean',
    'matrix_gpu-217_time_mean', 'matrix_cpu-95_lastday_mean', 'matrix_gpu-226_type-163_max', 'matrix_123_temporal_20_lastday_sum', 'matrix_gpu-217_type-163_sum',
    'matrix_123_type0.590.5_20_type-163_mean', 'matrix_gpu-226_pos-log_max', 'matrix_123_type136_20_max', 'matrix_gpu-226_time_max', 'matrix_gpu-217_lastday_mean', 'matrix_cpu-99_type-163_sum', 'matrix_cpu-99_pos-log_sum', 'matrix_gpu-700_type-163_max', 'matrix_gpu-217_time_max'
][:100]

In [ ]:
class Config:
    seed = 42
    version = VERSION
    
    folds_file = "../input/folds_4.csv"
    k = 4
    mode = ""

    features = FEATURES
    features = [ft for ft in features if ft not in TO_REMOVE]

    cat_features = []

    target = TARGET
    pos_ratio = POS_RATIO

    use_gt_sessions = True  # filter out sessions with no gt
    use_gt_pos = False  # add candidates from gt
    gt_regex = ""
    
    model = "xgb"

    params = PARAMS[model]

    use_es = True
    num_boost_round = 10000
    
    probs_file = None  # PROBS_PATHS[target]
    probs_mode = ""  # "head"  "rank_40"
    restrict_all = False

    selected_folds = [0, 1, 2, 3]
    folds_optimize = [0, 1, 2, 3]
    n_trials = 20

    pca_components = 0
    
    use_extra = False
    extra_regex = EXTRA_REGEX
    extra_prop = 0.

### Main

In [ ]:
DEBUG = False
DEBUG_MORE = False

In [ ]:
%%time

log_folder = None
run = None
if not DEBUG:
    log_folder = prepare_log_folder(LOG_PATH)
    run = init_neptune(Config, log_folder)
    print(f'Logging results to {log_folder}')
    create_logger(directory=log_folder, name="logs.txt")

    save_config(Config, log_folder + 'config')

ft_imp = kfold(REGEX, TEST_REGEX, Config, log_folder=log_folder, debug=DEBUG_MORE, run=run)

if run is not None:
    run.stop()

Done